In [0]:
!pip install soundfile

In [0]:
# importing the required libraries
import os
from scipy.io import wavfile
import numpy as np
import scipy
import scipy.signal
import soundfile as sf
import random
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import csv
from collections import namedtuple
import datetime
import time
from google.colab import drive
import h5py
import cv2
from PIL import Image, ImageOps

In [0]:
drive.mount('/content/drive')
os.chdir('/content/drive/') #connecting to google drive

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
# reading the boxes and storing that information

labelFile=("Shared drives/Stethoscope for the rainforests/Training data set /Table_13AB_v2.txt")
times = [] 
freqs = []
files = []
sonotypes = []
actTimes = []

with open(labelFile) as file:
  label_reader = csv.reader(file, delimiter='\t')
  for row in label_reader:
    # not use the first row
    if (not row[0].isnumeric()):
      continue

    # all the recording are either start at 0min or 30mins
    duration = float(row[4]) - float(row[3])
    timeWeight = [3600,60,1]
    actSta = sum([a*b for a,b in zip(timeWeight, map(int,row[10].replace('.',":").split(':')[:3]))])
    actEnd = actSta + duration

    fileName = row[7].split("\\")[-1]
    
    # get the start time of cur recording
    # 3B_20180607_070500_Dawn_Sunrise_Based.wav
    # or 20180908_060000_13A_24H [1.4452 116.9811].wav
    splName = fileName.split("_")

    # actual time is either in index 1 or 2 in splName
    if (len(splName[1]) == 6):
      strTime = splName[1]         
    elif (len(splName[2]) == 6):   
      strTime = splName[2]
    else: 
      print(row[0])
      continue  

    recLis = [int(strTime[0]) * 10 + int(strTime[1]),
              int(strTime[2]) * 10 + int(strTime[3]), 
              int(strTime[4]) * 10 + int(strTime[5])]    

    recSta = sum([a*b for a,b in zip(timeWeight, recLis)])

    # start, end time used to corp the specs
    start = actSta - recSta
    end = start + duration

    try:
      sonotypes.append(int(row[11]))
      times.append([start,end])
      actTimes.append([actSta, actEnd])
      freqs.append([float(row[5]), float(row[6])])
      files.append(fileName) 
    except:
      # selection number
      print(row[0])
      pass

print("dataset size:  %i" % len(times))    
print("filename sample: " + files[0])

print(files[len(files)-1])
print(actTimes[len(files)-1])
print(times[len(files)-1])

dataset size:  1651
filename sample: 3B_20180607_070500_Dawn_Sunrise_Based.wav
20190406_060000_13A_24H [1.4438 116.9805].wav
[23175, 23198.148843195988]
[1575, 1598.1488431959879]


In [0]:
# processing the signal using python library librosa and converted it 
# 1-min spectrograms using short-time fourier transform

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import librosa.display
import math
fileDir = "Shared drives/Stethoscope for the rainforests/Sound data/EastKalimantan/13AB/"
mydir = "/content/drive/My Drive/Stethoscope/min-spectrogram/"


soundFiles = os.listdir(fileDir)
usedFiles = np.intersect1d(soundFiles, files)

# ----------- Debug the memory leak --------------------------

for curFile in usedFiles[]:
  print("processing " + curFile)
  filePath=fileDir+curFile
  filename = curFile[:-4]
  y, sr = librosa.load(filePath)
  length_of_audio = librosa.get_duration(y=y, sr=sr)
  final_counter = math.floor(length_of_audio/60)
  for i in range(final_counter):
    y, sr = librosa.load(filePath, offset=60*i, duration=60)
    audio, _ = librosa.effects.trim(y)
    hop_length = 512
    n_fft = 2048
    D = np.abs(librosa.stft(audio, n_fft=n_fft,  
                        hop_length=hop_length))
    plt.figure()
    DB = librosa.amplitude_to_db(D, ref=np.max)
    librosa.display.specshow(DB, sr=sr, hop_length=hop_length, 
                         x_axis='time', y_axis='linear');
    plt.axis('off')
    plt.savefig(mydir + filename + "_" + str(i) + ".jpg", bbox_inches='tight',pad_inches = 0)
    plt.close()
    #plt.colorbar(format='%+2.0f dB');

processing 20190606_060400_Dawn_Sunrise_Based [13B_.4444 116.9808].wav
processing 20190606_174600_Dusk_SunsetBased [13B_.4444 116.9808].wav
processing 3B_20180607_070500_Dawn_Sunrise_Based.wav


### Additional Code

In [0]:
# conversion to melspectrograms

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import librosa.display
import math
fileDir = "Shared drives/Stethoscope for the rainforests/Sound data/EastKalimantan/13AB/"
mydir = "/content/drive/My Drive/Stethoscope/min-spectrogram/"


soundFiles = os.listdir(fileDir)
usedFiles = np.intersect1d(soundFiles, files)

for curFile in usedFiles:
  filePath=fileDir+curFile
  y, sr = librosa.load(filePath)
  length_of_audio = librosa.get_duration(y=y, sr=sr)
  final_counter = math.floor(length_of_audio/60)
  for i in range(final_counter):
    print("processing " + curFile)
    y, sr = librosa.load(filePath, offset=60*i, duration=60) #offset=,duration=40
    S = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=128,)
    plt.figure(figsize=(10, 6.67))
    S_dB = librosa.power_to_db(S, ref=np.max)
    librosa.display.specshow(S_dB, x_axis='time',
                          y_axis='linear', sr=sr
                          )
    filename = curFile[:-4]
    plt.savefig(mydir + filename + "_" + str(i) + ".jpg")
    plt.close()

In [0]:
# conversion to melspectrograms

fileDir = "Shared drives/Stethoscope for the rainforests/Sound data/EastKalimantan/13AB/"
soundFiles = os.listdir(fileDir)
usedFiles = np.intersect1d(soundFiles, files)

for curFile in usedFiles:
  print("processing " + curFile)

  for i in range(len(files)):
    if(files[i] == curFile):
      startTime = times[i][0]
      duration = times[i][1] - startTime
      freqBox = freqs[i]

  filePath = fileDir + curFile
  y, sr = librosa.load(filePath,offset=startTime,duration=duration)
  S = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=128,)
  plt.figure(figsize=(10, 4))
  S_dB = librosa.power_to_db(S, ref=np.max)
  librosa.display.specshow(S_dB, x_axis='time',
                          y_axis='linear', sr=sr
                          )
  plt.savefig("/content/drive/My Drive/Stethoscope/spectrogram/" + curFile + ".jpg")

processing 20180607_060400_Dawn_Sunrise____13B_.4455 116.9813].wav
processing 20180607_120400_Midday_Sunrise___13B_.4455 116.9813].wav
